# Generate Color Bins For Each Layer

### 0. Import libraries and create empty dictionaries

In [1]:
import geopandas as gpd
import numpy as np
import mapclassify as mc
import matplotlib.pyplot as plt

In [2]:
total_dict = {}
county_dict = {}
state_dict = {}
dph_illinois_dict = {}
who_world_dict = {}
vul_dict = {}

In [3]:
dynamic_dict = {}
dynamic_county_dict = {}
dynamic_state_dict = {}
dynamic_illinois_dict = {}

### 1. For US County-level Data

#### 1.0 Load Data

In [61]:
counties_gdf = gpd.read_file('nyt_counties_data.geojson')

In [62]:
counties_gdf.head(5)

,NAME,state_name,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,start,end,dt_unit,geometry
0,Bladen,North Carolina,33778,37017,2020-04-03,2020-05-06,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",528.0,5.0,5.0,0.0,2020-01-21,2020-07-26,day,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ..."
1,Stanly,North Carolina,61114,37167,2020-03-20,2020-04-09,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",778.0,14.0,6.0,0.0,2020-01-21,2020-07-26,day,"MULTIPOLYGON (((-80.49738 35.20210, -80.29542 ..."
2,Summit,Ohio,541810,39153,2020-03-14,2020-03-27,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2914.0,35.0,212.0,0.0,2020-01-21,2020-07-26,day,"MULTIPOLYGON (((-81.68699 41.13596, -81.68495 ..."
3,Sullivan,Pennsylvania,6177,42113,2020-04-04,None,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",10.0,0.0,0.0,0.0,2020-01-21,2020-07-26,day,"MULTIPOLYGON (((-76.81373 41.59003, -76.22014 ..."
4,Upshur,Texas,40769,48459,2020-03-20,2020-07-25,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",161.0,3.0,1.0,0.0,2020-01-21,2020-07-26,day,"MULTIPOLYGON (((-95.15274 32.66095, -95.15211 ..."


In [63]:
case = counties_gdf['today_case']
death = counties_gdf['today_death']
case_per_100k_capita = counties_gdf['today_case']/(counties_gdf['population']/100000)
death_per_100k_capita = counties_gdf['today_death']/(counties_gdf['population']/100000)
# death_case_ratio = counties_gdf['today_death']/counties_gdf['today_case']
# death_case_ratio = death_case_ratio.replace(np.nan, 0)

In [64]:
log_case = case.apply(lambda x: np.log(x + 1))
log_death = death.apply(lambda x: np.log(x + 1))
log_case_per_100k_capita = case_per_100k_capita.apply(lambda x: np.log(x + 1))
log_death_per_100k_capita = death.apply(lambda x: np.log(x + 1))

#### 1.1 Get Constant Classes from today's data

In [4]:
def create_dict(column):
    tmp_dict = {}
    tmp_dict['Quantiles'] = {
        'bins': ','.join(mc.Quantiles(column, k=6).bins.astype(str)),
        'k': 6
    }
    tmp_dict['FisherJenks'] = {
        'bins': ','.join(mc.FisherJenks(column, k=6).bins.astype(str)),
        'k': 6
    }
    tmp_dict['NaturalBreaks'] = {
        'bins': ','.join(mc.NaturalBreaks(column, k=6).bins.astype(str)),
        'k': 6
    }
    return tmp_dict

In [66]:
county_dict['case'] = {}
county_dict['death'] = {}
county_dict['case_per_100k_capita'] = {}
county_dict['death_per_100k_capita'] = {}

In [67]:
county_dict['case']['nolog'] = create_dict(case)
county_dict['case']['log'] = create_dict(log_case)
county_dict['death']['nolog'] = create_dict(death)
county_dict['death']['log'] = create_dict(log_death)
county_dict['case_per_100k_capita']['nolog'] = create_dict(case_per_100k_capita)
county_dict['case_per_100k_capita']['log'] = create_dict(log_case_per_100k_capita)
county_dict['death_per_100k_capita']['nolog'] = create_dict(death_per_100k_capita)
county_dict['death_per_100k_capita']['log'] = create_dict(log_death_per_100k_capita)


C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 5
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 5
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in

In [68]:
county_dict

{'case': {'nolog': {'Quantiles': {'bins': '20.0,65.0,151.0,349.0,1057.0,228445.0',
    'k': 6},
   'FisherJenks': {'bins': '3475.0,11780.0,27077.0,65349.0,108677.0,228445.0',
    'k': 6},
   'NaturalBreaks': {'bins': '3501.0,11780.0,27077.0,65349.0,108677.0,228445.0',
    'k': 6}},
  'log': {'Quantiles': {'bins': '3.044522437723423,4.189654742026425,5.0238805208462765,5.857933154483459,6.964135612418245,12.339055137536311',
    'k': 6},
   'FisherJenks': {'bins': '1.3862943611198906,3.295836866004329,4.727387818712341,6.0844994130751715,7.791109510610028,12.339055137536311',
    'k': 6},
   'NaturalBreaks': {'bins': '1.3862943611198906,3.295836866004329,4.736198448394496,6.0844994130751715,7.791109510610028,12.339055137536311',
    'k': 6}}},
 'death': {'nolog': {'Quantiles': {'bins': '0.0,2.0,6.0,25.0,22956.0', 'k': 6},
   'FisherJenks': {'bins': '109.0,456.0,1242.0,2784.0,4830.0,22956.0', 'k': 6},
   'NaturalBreaks': {'bins': '113.0,470.0,1242.0,2784.0,4830.0,22956.0',
    'k': 6}},


### 2. For US State-level Data

#### 2.0 Load Data

In [74]:
state_gdf = gpd.read_file('nyt_states_data.geojson')

In [75]:
state_gdf.head(5)

,NAME,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,start,end,dt_unit,geometry
0,West Virginia,1829054,54,2020-03-17,2020-03-29,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",5960.0,139.0,103.0,0.0,2020-01-21,2020-07-26,day,"POLYGON ((-80.51899 40.63880, -80.51934 39.721..."
1,Florida,20598139,12,2020-03-01,2020-03-06,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",423847.0,9344.0,5853.0,77.0,2020-01-21,2020-07-26,day,"MULTIPOLYGON (((-82.98748 24.62538, -82.89993 ..."
2,Illinois,12821497,17,2020-01-24,2020-03-17,"0,0,0,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",173079.0,1410.0,7595.0,2.0,2020-01-21,2020-07-26,day,"POLYGON ((-90.64284 42.50848, -87.01994 42.493..."
3,Minnesota,5527358,27,2020-03-06,2020-03-21,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",51193.0,862.0,1614.0,3.0,2020-01-21,2020-07-26,day,"POLYGON ((-97.22872 49.00056, -95.15371 48.998..."
4,Maryland,6003435,24,2020-03-05,2020-03-18,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",84306.0,689.0,3440.0,7.0,2020-01-21,2020-07-26,day,"POLYGON ((-77.71952 39.32131, -77.76086 39.340..."


#### 2.1 Get Constant Classes from today's data

In [77]:
state_case = state_gdf['today_case']
state_death = state_gdf['today_death']
state_case_per_100k_capita = state_gdf['today_case']/(state_gdf['population']/100000)
state_death_per_100k_capita = state_gdf['today_death']/(state_gdf['population']/100000)
#death_case_ratio = counties_gdf['today_death']/counties_gdf['today_case']
#death_case_ratio = death_case_ratio.replace(np.nan, 0)

In [78]:
state_death_per_100k_capita = state_death_per_100k_capita.replace(np.nan,0)
state_death_per_100k_capita = state_death_per_100k_capita.replace(np.inf,0)
state_case_per_100k_capita = state_case_per_100k_capita.replace(np.nan,0)
state_case_per_100k_capita = state_case_per_100k_capita.replace(np.inf,0)

In [79]:
state_log_case = state_case.apply(lambda x: np.log(x + 1))
state_log_death = state_death.apply(lambda x: np.log(x + 1))
state_log_case_per_100k_capita = state_case_per_100k_capita.apply(lambda x: np.log(x + 1))
state_log_death_per_100k_capita =state_death_per_100k_capita.apply(lambda x: np.log(x + 1))

In [80]:
state_dict['case'] = {}
state_dict['death'] = {}
state_dict['case_per_100k_capita'] = {}
state_dict['death_per_100k_capita'] = {}

In [81]:
state_dict['case']['nolog'] = create_dict(state_case)
state_dict['case']['log'] = create_dict(state_log_case)
state_dict['death']['nolog'] = create_dict(state_death)
state_dict['death']['log'] = create_dict(state_log_death)
state_dict['case_per_100k_capita']['nolog'] = create_dict(state_case_per_100k_capita)
state_dict['case_per_100k_capita']['log'] = create_dict(state_log_case_per_100k_capita)
state_dict['death_per_100k_capita']['nolog'] = create_dict(state_death_per_100k_capita)
state_dict['death_per_100k_capita']['log'] = create_dict(state_log_death_per_100k_capita)

In [82]:
state_dict

{'case': {'nolog': {'Quantiles': {'bins': '5880.000000000001,18262.000000000004,42919.0,79294.00000000001,112881.00000000001,459338.0',
    'k': 6},
   'FisherJenks': {'bins': '27682.0,63881.0,91330.0,115637.0,181283.0,459338.0',
    'k': 6},
   'NaturalBreaks': {'bins': '27682.0,63881.0,91330.0,115637.0,181283.0,459338.0',
    'k': 6}},
  'log': {'Quantiles': {'bins': '8.679482094459956,9.812632434192041,10.667093196744634,11.280930353932701,11.63409830430028,13.037543778531418',
    'k': 6},
   'FisherJenks': {'bins': '5.8916442118257715,8.246695943718557,9.855032332266724,11.06479291054266,12.107820141331725,13.037543778531418',
    'k': 6},
   'NaturalBreaks': {'bins': '5.8916442118257715,8.769817872052617,10.228573785361645,11.06479291054266,12.107820141331725,13.037543778531418',
    'k': 6}}},
 'death': {'nolog': {'Quantiles': {'bins': '103.00000000000003,401.00000000000006,901.0,1795.0,4413.0000000000055,32305.0',
    'k': 6},
   'FisherJenks': {'bins': '829.0,2078.0,5090.0,852

### 3. For IDPH County-level Data

#### 3.0 Load Data

In [4]:
dph_illinois_gdf = gpd.read_file('illinois/dph_county_data.geojson')

In [5]:
dph_illinois_gdf.head(5)

,id,NAME,cases_ts,dt_first_case,today_case,today_new_case,deaths_ts,dt_first_death,today_death,today_new_death,today_tested,today_new_tested,start,end,dt_unit,population,change_ts,geometry
0,Chicago,Chicago,"0,0,0,0,0,519,519,782,915,1161,1364,1610,2026,...",2020-03-22,318604,263,"0,0,0,0,0,4,4,5,5,9,9,12,16,16,21,39,41,57,76,...",2020-03-22,5785,1,6215968,18185,2020-03-17,2021-10-04,day,2693976,"0,0,0,0,0,0,0,0,0,0,0,0,0,2.24,1.44,1.37,0.82,...","MULTIPOLYGON (((-87.93514 42.00089, -87.93521 ..."
1,McHenry,McHenry,"2,4,6,8,11,12,12,14,19,27,45,47,52,52,63,69,81...",2020-03-17,33781,43,"0,0,0,0,0,0,0,0,0,1,1,2,2,2,2,2,3,3,3,3,3,4,5,...",2020-03-26,320,0,491586,1836,2020-03-17,2021-10-04,day,308570,"0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,3.08,2.33,1.57,0...","POLYGON ((-88.70742 42.49352, -88.70741 42.493..."
2,Boone,Boone,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,4,6,7,...",2020-04-05,7728,6,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...",2020-04-08,83,0,83383,165,2020-03-17,2021-10-04,day,53577,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,4.0,...","POLYGON ((-88.70742 42.49352, -88.70750 42.493..."
3,Ogle,Ogle,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,4,5,7,10,1...",2020-03-31,7331,7,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-04-15,87,0,85426,326,2020-03-17,2021-10-04,day,50923,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,1.0,4....","POLYGON ((-89.68809 42.19950, -89.68807 42.184..."
4,Will,Will,"2,3,9,10,12,21,21,28,40,67,104,127,182,223,228...",2020-03-17,89027,122,"0,0,0,1,1,1,1,1,2,3,3,4,4,5,8,9,8,10,11,12,19,...",2020-03-20,1105,1,1377062,4447,2020-03-17,2021-10-04,day,692310,"0,0,0,0,0,0,0,0,0,0,0,0,0,9.63,6.69,6.62,4.47,...","POLYGON ((-88.26146 41.72439, -88.26103 41.708..."


In [6]:
dph_illinois_gdf = dph_illinois_gdf[dph_illinois_gdf['id'] != 'Illinois']

#### 3.1 Get Constant Classes from today's data

In [7]:
illinois_case = dph_illinois_gdf['today_case']
illinois_death = dph_illinois_gdf['today_death']
illinois_case_per_100k_capita = dph_illinois_gdf['today_case']/(dph_illinois_gdf['population']/100000)
illinois_death_per_100k_capita = dph_illinois_gdf['today_death']/(dph_illinois_gdf['population']/100000)
#death_case_ratio = counties_gdf['today_death']/counties_gdf['today_case']
#death_case_ratio = death_case_ratio.replace(np.nan, 0)

In [8]:
illinois_death_per_100k_capita = illinois_death_per_100k_capita.replace(np.nan,0)
illinois_death_per_100k_capita = illinois_death_per_100k_capita.replace(np.inf,0)
illinois_case_per_100k_capita = illinois_case_per_100k_capita.replace(np.nan,0)
illinois_case_per_100k_capita = illinois_case_per_100k_capita.replace(np.inf,0)

In [9]:
illinois_log_case = illinois_case.apply(lambda x: np.log(x + 1))
illinois_log_death = illinois_death.apply(lambda x: np.log(x + 1))
illinois_log_case_per_100k_capita = illinois_case_per_100k_capita.apply(lambda x: np.log(x + 1))
illinois_log_death_per_100k_capita =illinois_death_per_100k_capita.apply(lambda x: np.log(x + 1))

In [10]:
dph_illinois_dict['case'] = {}
dph_illinois_dict['death'] = {}
dph_illinois_dict['case_per_100k_capita'] = {}
dph_illinois_dict['death_per_100k_capita'] = {}

In [13]:
dph_illinois_dict['case']['nolog'] = create_dict(illinois_case)
dph_illinois_dict['case']['log'] = create_dict(illinois_log_case)
dph_illinois_dict['death']['nolog'] = create_dict(illinois_death)
dph_illinois_dict['death']['log'] = create_dict(illinois_log_death)
dph_illinois_dict['case_per_100k_capita']['nolog'] = create_dict(illinois_case_per_100k_capita)
dph_illinois_dict['case_per_100k_capita']['log'] = create_dict(illinois_log_case_per_100k_capita)
dph_illinois_dict['death_per_100k_capita']['nolog'] = create_dict(illinois_death_per_100k_capita)
dph_illinois_dict['death_per_100k_capita']['log'] = create_dict(illinois_log_death_per_100k_capita)

In [14]:
dph_illinois_dict

{'case': {'nolog': {'Quantiles': {'bins': '1842.0000000000005,2493.0000000000005,4236.0,6705.0,15818.00000000002,318604.0',
    'k': 6},
   'FisherJenks': {'bins': '8090,22418,39940,78468,105694,318604', 'k': 6},
   'NaturalBreaks': {'bins': '8090.0,22418.0,39940.0,78468.0,105694.0,318604.0',
    'k': 6}},
  'log': {'Quantiles': {'bins': '7.519149957669823,7.821643126239982,8.351610750626559,8.810757927026176,9.668967028198582,12.671707369994703',
    'k': 6},
   'FisherJenks': {'bins': '7.224753405767971,8.200837258379849,8.99850761180784,10.18074042949737,11.568312867049787,12.671707369994703',
    'k': 6},
   'NaturalBreaks': {'bins': '7.224753405767971,8.200837258379849,8.99850761180784,10.18074042949737,11.568312867049787,12.671707369994703',
    'k': 6}}},
 'death': {'nolog': {'Quantiles': {'bins': '21.000000000000007,39.00000000000001,68.0,98.0,214.00000000000034,5785.0',
    'k': 6},
   'FisherJenks': {'bins': '150,365,861,1366,5281,5785', 'k': 6},
   'NaturalBreaks': {'bins': 

### 4. For IDPH Zipcode-level Case (Static)

#### 4.0 Load Data

In [15]:
# dph_zipcode_gdf = gpd.read_file('illinois/dph_zipcode_data.geojson')

In [16]:
# dph_zipcode_gdf

,id,confirmed_cases,total_tested,geometry
0,62359,21.0,297.0,"POLYGON ((-91.18290 40.02688, -91.17656 40.020..."
1,62376,204.0,2040.0,"POLYGON ((-91.35721 40.17576, -91.35600 40.151..."
2,62312,377.0,4169.0,"POLYGON ((-90.97797 39.78881, -90.97787 39.772..."
3,62365,109.0,936.0,"POLYGON ((-91.16584 39.83701, -91.14703 39.837..."
4,62246,1530.0,35518.0,"POLYGON ((-89.56254 38.91811, -89.56794 38.923..."
...,...,...,...,...
1379,60484,744.0,11341.0,"POLYGON ((-87.70969 41.45644, -87.70949 41.456..."
1380,62948,2225.0,32799.0,"MULTIPOLYGON (((-89.02159 37.74516, -89.03561 ..."
1381,62921,52.0,643.0,"POLYGON ((-89.09155 37.79258, -89.08896 37.807..."
1382,62915,107.0,1218.0,"POLYGON ((-89.11994 37.78039, -89.12263 37.781..."


#### 4.1 Get Constant Classes from today's data

In [17]:
# dph_zipcode_tested = dph_zipcode_gdf['total_tested']
# dph_zipcode_case = dph_zipcode_gdf['confirmed_cases']

In [18]:
# dph_zipcode_log_tested = dph_zipcode_tested.apply(lambda x: np.log(x + 1))
# dph_zipcode_log_case = dph_zipcode_case.apply(lambda x: np.log(x + 1))

In [19]:
# dph_illinois_dict['tested'] = {}
# dph_illinois_dict['zipcode_case'] = {}
# dph_illinois_dict['zipcode_tested'] = {}

In [20]:
# dph_illinois_dict['zipcode_case']['nolog'] = create_dict(dph_zipcode_case)
# dph_illinois_dict['zipcode_case']['log'] = create_dict(dph_zipcode_log_case)
# dph_illinois_dict['zipcode_tested']['nolog'] = create_dict(dph_zipcode_tested)
# dph_illinois_dict['zipcode_tested']['log'] = create_dict(dph_zipcode_log_tested)

In [21]:
# dph_illinois_dict

{'case': {'nolog': {'Quantiles': {'bins': '1842.0000000000005,2493.0000000000005,4236.0,6705.0,15818.00000000002,318604.0',
    'k': 6},
   'FisherJenks': {'bins': '8090,22418,39940,78468,105694,318604', 'k': 6},
   'NaturalBreaks': {'bins': '8090.0,22418.0,39940.0,78468.0,105694.0,318604.0',
    'k': 6}},
  'log': {'Quantiles': {'bins': '7.519149957669823,7.821643126239982,8.351610750626559,8.810757927026176,9.668967028198582,12.671707369994703',
    'k': 6},
   'FisherJenks': {'bins': '7.224753405767971,8.200837258379849,8.99850761180784,10.18074042949737,11.568312867049787,12.671707369994703',
    'k': 6},
   'NaturalBreaks': {'bins': '7.224753405767971,8.200837258379849,8.99850761180784,10.18074042949737,11.568312867049787,12.671707369994703',
    'k': 6}}},
 'death': {'nolog': {'Quantiles': {'bins': '21.000000000000007,39.00000000000001,68.0,98.0,214.00000000000034,5785.0',
    'k': 6},
   'FisherJenks': {'bins': '150,365,861,1366,5281,5785', 'k': 6},
   'NaturalBreaks': {'bins': 

### 5. For WHO World Data

#### 5.0 Load Data

In [100]:
who_world_gdf = gpd.read_file('worldwide/who_world_data.geojson')

#### 5.1 Get Constant Classes from today's data

In [101]:
who_world_case = who_world_gdf['today_case']
who_world_death = who_world_gdf['today_death']
who_world_case_per_100k_capita = who_world_gdf['today_case']/(who_world_gdf['population']/100000)
who_world_death_per_100k_capita = who_world_gdf['today_death']/(who_world_gdf['population']/100000)

In [102]:
who_world_log_case = who_world_case.apply(lambda x: np.log(x + 1))
who_world_log_death = who_world_death.apply(lambda x: np.log(x + 1))
who_world_log_case_per_100k_capita = who_world_case_per_100k_capita.apply(lambda x: np.log(x + 1))
who_world_log_death_per_100k_capita = who_world_death_per_100k_capita.apply(lambda x: np.log(x + 1))

In [103]:
who_world_dict['case'] = {}
who_world_dict['death'] = {}
who_world_dict['case_per_100k_capita'] = {}
who_world_dict['death_per_100k_capita'] = {}

In [104]:
who_world_dict['case']['nolog'] = create_dict(who_world_case)
who_world_dict['case']['log'] = create_dict(who_world_log_case)
who_world_dict['death']['nolog'] = create_dict(who_world_death)
who_world_dict['death']['log'] = create_dict(who_world_log_death)
who_world_dict['case_per_100k_capita']['nolog'] = create_dict(who_world_case_per_100k_capita)
who_world_dict['case_per_100k_capita']['log'] = create_dict(who_world_log_case_per_100k_capita)
who_world_dict['death_per_100k_capita']['nolog'] = create_dict(who_world_death_per_100k_capita)
who_world_dict['death_per_100k_capita']['log'] = create_dict(who_world_log_death_per_100k_capita)

In [105]:
who_world_dict

{'case': {'nolog': {'Quantiles': {'bins': '19.66666666666667,352.6666666666667,2058.0,9423.666666666672,50798.50000000012,4148011.0',
    'k': 6},
   'FisherJenks': {'bins': '113556.0,445433.0,818120.0,1435453.0,2394513.0,4148011.0',
    'k': 6},
   'NaturalBreaks': {'bins': '113556.0,445433.0,818120.0,1435453.0,2394513.0,4148011.0',
    'k': 6}},
  'log': {'Quantiles': {'bins': '3.0278418612972597,5.868340600805565,7.629907769568833,9.15090090956101,10.835466980983195,15.238139741287833',
    'k': 6},
   'FisherJenks': {'bins': '1.6094379124341003,4.770684624465665,7.106606137727303,9.343559215762406,11.640060192485223,15.238139741287833',
    'k': 6},
   'NaturalBreaks': {'bins': '1.6094379124341003,4.770684624465665,7.106606137727303,9.343559215762406,11.640060192485223,15.238139741287833',
    'k': 6}}},
 'death': {'nolog': {'Quantiles': {'bins': '0.0,8.666666666666671,41.5,155.33333333333334,1174.0000000000055,145727.0',
    'k': 6},
   'FisherJenks': {'bins': '2928.0,18030.0,3510

### 6. For Vulnerability Data

#### 6.0 Load Data

In [8]:
vul_gdf = gpd.read_file('illinois/vulnerability.geojson')

#### 6.1 Get Constant Classes from today's data

In [9]:
vul = vul_gdf['today_vul']

In [10]:
vul_dict['case'] = {}

In [11]:
vul_dict['case']['nolog'] = create_dict(vul)

In [12]:
vul_dict

{'case': {'nolog': {'Quantiles': {'bins': '0.309049268215048,0.365615406049071,0.417252086664159,0.4725948884265381,0.5381530696760801,0.717163598455802',
    'k': 6},
   'FisherJenks': {'bins': '0.28182576656381,0.346748353852654,0.409489937192436,0.474170519727913,0.543735718587675,0.717163598455802',
    'k': 6},
   'NaturalBreaks': {'bins': '0.283040610526724,0.346748353852654,0.408732752726884,0.473144761690751,0.543375327243225,0.717163598455802',
    'k': 6}}}}

### 7. Summary and ouput

In [111]:
import json
total_dict['county'] = county_dict
total_dict['state'] = state_dict
total_dict['dph_illinois'] = dph_illinois_dict
total_dict['who_world'] = who_world_dict
total_dict['vulnerability'] = vul_dict
# dynamic_dict['county'] = dynamic_county_dict
# dynamic_dict['state'] = dynamic_state_dict
# dynamic_dict['illinois'] = dynamic_illinois_dict
with open('classes.json','w') as json_file:
    json.dump(total_dict, json_file)
# with open('dynamic_classes.json','w') as json_file:
#     json.dump(dynamic_dict, json_file)
print('done')

done
